<a href="https://colab.research.google.com/github/Yukineri/MechineLearning/blob/main/Fakenews_Detection_July_December_2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instal TensorFlow jika belum terinstal
!pip install tensorflow

In [ ]:
# Instal library gdown dan import library yang diperlukan
!pip install gdown
from google.colab import drive
import gdown
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#gunakan id file dari Google Drive
fake_file_id = '1SAfl5mgo_s2PT6d7w1hjlDEJxoa6L3FH'
true_file_id = '16-xIRRVavPRNkPu85zeSwiQEZXqai1YD'

In [ ]:
# Paths
fake_output_path = '/content/drive/MyDrive/fake_dataset.csv'
true_output_path = '/content/drive/MyDrive/true_dataset.csv'

In [ ]:
# Unduh dataset Fake
fake_url = f'https://drive.google.com/uc?id={fake_file_id}'
gdown.download(fake_url, fake_output_path, quiet=False)

# Unduh dataset True
true_url = f'https://drive.google.com/uc?id={true_file_id}'
gdown.download(true_url, true_output_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1SAfl5mgo_s2PT6d7w1hjlDEJxoa6L3FH
To: /content/drive/MyDrive/fake_dataset.csv
100%|██████████| 2.50M/2.50M [00:00<00:00, 123MB/s]
Downloading...
From: https://drive.google.com/uc?id=16-xIRRVavPRNkPu85zeSwiQEZXqai1YD
To: /content/drive/MyDrive/true_dataset.csv
100%|██████████| 7.39M/7.39M [00:00<00:00, 17.9MB/s]


'/content/drive/MyDrive/true_dataset.csv'

In [ ]:
# Baca dataset Fake dan True
fake_dataset = pd.read_csv(fake_output_path)
true_dataset = pd.read_csv(true_output_path)

In [ ]:
# Tambahkan label (0 untuk fake news, 1 untuk true news)
fake_dataset['label'] = 0
true_dataset['label'] = 1

In [ ]:
# Gabungkan kedua dataset
combined_data = pd.concat([fake_dataset, true_dataset], ignore_index=True)

In [ ]:
# Pilih kolom-kolom yang diperlukan (sesuaikan dengan struktur dataset)
combined_data = combined_data[['title', 'text', 'subject', 'date', 'label']]

In [ ]:
# Bagi dataset menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(
    combined_data[['title', 'text', 'subject', 'date']], combined_data['label'], test_size=0.2, random_state=42
)

In [ ]:
# Gabungkan title dan text menjadi satu kolom
X_train['combined_text'] = X_train['title'] + ' ' + X_train['text']
X_test['combined_text'] = X_test['title'] + ' ' + X_test['text']

In [ ]:
# Proses teks dengan TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train['combined_text'])
X_test_tfidf = vectorizer.transform(X_test['combined_text'])

In [ ]:
# Bangun model dengan TensorFlow
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=X_train_tfidf.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Latih model
model.fit(X_train_tfidf.toarray(), y_train, epochs=5, batch_size=64)

Epoch 1/5
49/49 [==============================] - 642s 13s/step - loss: 0.5643 - accuracy: 0.7577
Epoch 2/5
49/49 [==============================] - 644s 13s/step - loss: 0.5578 - accuracy: 0.7571
Epoch 3/5
49/49 [==============================] - 630s 13s/step - loss: 0.5578 - accuracy: 0.7571
Epoch 4/5
49/49 [==============================] - 634s 13s/step - loss: 0.5561 - accuracy: 0.7571
Epoch 5/5
49/49 [==============================] - 651s 13s/step - loss: 0.5579 - accuracy: 0.7571


In [ ]:
# Evaluasi model
y_pred_prob = model.predict(X_test_tfidf.toarray())
y_pred = (y_pred_prob > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

25/25 [==============================] - 49s 2s/step
Accuracy: 0.7377892030848329
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       204
           1       0.74      1.00      0.85       574

    accuracy                           0.74       778
   macro avg       0.37      0.50      0.42       778
weighted avg       0.54      0.74      0.63       778



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
